In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
model_path = '/content/drive/MyDrive/distilbert-base-uncased-finetuned/checkpoint-2000'

In [3]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,499 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package git-lfs.
(Reading database ... 155222 files and directories c

In [4]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 8.6 MB/s 
     |████████████████████████████████| 61 kB 689 kB/s 
     |████████████████████████████████| 596 kB 63.8 MB/s 
     |████████████████████████████████| 3.3 MB 57.0 MB/s 
     |████████████████████████████████| 895 kB 74.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
!git config --global credential.helper store

In [6]:
from huggingface_hub import notebook_login

#notebook_login()


In [7]:
!pip install datasets

     |████████████████████████████████| 298 kB 6.9 MB/s 
     |████████████████████████████████| 132 kB 70.6 MB/s 
     |████████████████████████████████| 1.1 MB 63.9 MB/s 
     |████████████████████████████████| 243 kB 70.8 MB/s 
     |████████████████████████████████| 271 kB 75.0 MB/s 
     |████████████████████████████████| 160 kB 71.1 MB/s 
     |████████████████████████████████| 192 kB 54.1 MB/s 


In [8]:
from datasets import load_dataset, load_metric

In [9]:
base_url = 'https://raw.githubusercontent.com/ipavlopoulos/toxic_spans/master/data/'
datasets = load_dataset('csv', data_files={'train': base_url + 'tsd_train.csv', 'test': base_url + 'tsd_test.csv'})

Using custom data configuration default-73ce3741ec76b5f5


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/813k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/177k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-73ce3741ec76b5f5/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [10]:
#datasets['train']['text']

In [11]:
datasets

DatasetDict({
    train: Dataset({
        features: ['spans', 'text'],
        num_rows: 7939
    })
    test: Dataset({
        features: ['spans', 'text'],
        num_rows: 2000
    })
})

In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import spacy

In [13]:
nlp = spacy.load("en_core_web_sm")

In [14]:
# this will load the english tokenizer of SpaCy library to be applied on the sentence
nlp = spacy.load("en_core_web_sm")

def sentece_preprocess(sentece, span, nlp):
  """
  This function label tokens of the sentece based on its span
  Input:
  sentece [array]: the sentence that we want to labels its tokens as T(Toxic), and NT (Not toxic)
  span [str] : span in string format (Note: currently there is a bug for begining and end of that string 
  for special characters of '[', and ']' characters)
  nlp: SpaCy tokenizer for english sentece
  Return:
  An array of labels for the given sentence as T or NT that can be used for true labels of token classifications
  """
  doc = nlp(sentece)
  # token.idx returns the starting span of that token
  tokens = [(token, token.idx) for token in doc]
  tokens = np.array(tokens)
  processed_token = []
  toxic_word = []
  processed_sentence = []
  span_list = []
  for item in span.replace('[', ' ').replace(']', ',').split(','):
    try:
     span_list.append(int(item))
    except:
     pass
  span_list = np.array(span_list)
  for i in range(len(tokens)):
    processed_token.append(tokens[i][0])
    if tokens[i][1] in span_list:
      toxic_word.append(tokens[i][0])
  toxic_word = np.array(toxic_word)
  for i in range(len(tokens)):
    if tokens[i][0] in toxic_word:
      #processed_sentence.append('T')
      processed_sentence.append(int(1))
    else:
      #processed_sentence.append('NT')
      processed_sentence.append(int(0))
  #print(processed_sentence)
  # Pad the labels with max length
  #padded=pad_sequences([processed_sentence],padding="post",truncating="post",maxlen=192, value='NT', dtype = object)
  #flat_list = [item for sublist in padded for item in sublist]
  #processed_sentence = np.array(flat_list)
  processed_sentence = np.array(processed_sentence)
  #processed_token = np.array(processed_token)
  return processed_token, processed_sentence

In [15]:
import spacy
nlp = spacy.load("en_core_web_sm")

def sentence_labeler(df, nlp):
  labels = []
  tokens = []   
  for i in range(len(df)):
    temp_token, temp_label = sentece_preprocess(df['text'][i], df['spans'][i], nlp)
    labels.append(temp_label)
    tokens.append(temp_token)

  # converiting tokens to string
  token_str = []
  for item in tokens:
    test = []
    for element in item:
      test.append(str(element))
    token_str.append(test)
  # converiting labels to string
  label_str = []
  for item in labels:
    test_label = []
    for element in item:
      test_label.append(str(element))
    label_str.append(test_label)
  return token_str, labels

In [16]:
tokens, labels = sentence_labeler(datasets['train'], nlp)

In [17]:
datasets['train'] = datasets['train'].add_column('tokens', tokens)
datasets['train'] = datasets['train'].add_column('tags', labels)

In [18]:
datasets['train']

Dataset({
    features: ['spans', 'text', 'tokens', 'tags'],
    num_rows: 7939
})

In [19]:
tokens_test, labels_test = sentence_labeler(datasets['test'], nlp)

In [20]:
datasets['test'] = datasets['test'].add_column('tokens', tokens_test)
datasets['test'] = datasets['test'].add_column('tags', labels_test)

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")   


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [22]:
label_all_tokens = True

In [23]:
def tokenize_and_align_labels(examples):
    
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    
    labels = []
    for i, label in enumerate(examples["tags"]):
        if i==1000:
          print(label)
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        #print(word_ids)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            #print(word_idx)
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [24]:
pd.DataFrame(datasets['train']).dtypes

spans     object
text      object
tokens    object
tags      object
dtype: object

In [25]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [26]:
label_list = [0 , 1]
label_list

[0, 1]

In [27]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

#model = AutoModelForTokenClassification.from_pretrained("distilbert-base-uncased", num_labels=len(label_list))

In [28]:
model_checkpoint = "distilbert-base-uncased"

In [29]:
batch_size = 10

In [30]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"/content/drive/MyDrive/{model_name}-finetuned",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

In [31]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [32]:
!pip install seqeval

     |████████████████████████████████| 43 kB 1.5 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=c7706e6e1098cfa16139b6a9d3dd519efdb743c597be2af2113a82cbb736ccd4
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [33]:
metric = load_metric("seqeval")

Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

In [34]:
example = datasets["train"][4]

In [35]:
labels = [label_list[i] for i in example["tags"]]
metric.compute(predictions=[labels], references=[labels])

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:380: R

{'overall_accuracy': 1.0,
 'overall_f1': 0.0,
 'overall_precision': 0.0,
 'overall_recall': 0.0}

In [36]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [37]:
#model = AutoModelForTokenClassification.from_pretrained('/content/drive/MyDrive/distilbert-base-uncased-finetuned/checkpoint-2000')
model = AutoModelForTokenClassification.from_pretrained(model_path)

In [38]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [39]:
#trainer.train(model_path)
#trainer.train()

In [40]:
trainer.save_model()

Saving model checkpoint to /content/drive/MyDrive/distilbert-base-uncased-finetuned
Configuration saved in /content/drive/MyDrive/distilbert-base-uncased-finetuned/config.json
Model weights saved in /content/drive/MyDrive/distilbert-base-uncased-finetuned/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/distilbert-base-uncased-finetuned/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/distilbert-base-uncased-finetuned/special_tokens_map.json


In [41]:
!nvidia-smi

Thu Dec  9 00:52:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    33W / 250W |   1411MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [42]:
#!kill process_id

In [43]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens, spans, text, tags.
***** Running Prediction *****
  Num examples = 2000
  Batch size = 10


/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 0 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 1 seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/numpy/lib/function_base.py:380: R

{'overall_accuracy': 0.9677095822189431,
 'overall_f1': 0.0,
 'overall_precision': 0.0,
 'overall_recall': 0.0}

In [44]:
trainer.save_model("checkpoint-final")

Saving model checkpoint to checkpoint-final
Configuration saved in checkpoint-final/config.json
Model weights saved in checkpoint-final/pytorch_model.bin
tokenizer config file saved in checkpoint-final/tokenizer_config.json
Special tokens file saved in checkpoint-final/special_tokens_map.json


In [45]:
print(true_predictions[0])
print(true_labels[0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]


In [46]:
def f1(predictions, gold):
    """
    F1 (a.k.a. DICE) operating on two lists of offsets (e.g., character).
    >>> assert f1([0, 1, 4, 5], [0, 1, 6]) == 0.5714285714285714
    :param predictions: a list of predicted offsets
    :param gold: a list of offsets serving as the ground truth
    :return: a score between 0 and 1
    """
    if len(gold) == 0:
        return 1 if len(predictions)==0 else 0
    predictions1 = [tuple(l) for l in predictions]
    gold1 = [tuple(l) for l in gold]
    nom = 2*len(set(predictions1).intersection(set(gold1)))
    denom = len(set(predictions1))+len(set(gold1))
    return nom/denom

In [47]:
f1(true_predictions, true_labels)

0.5292348813899098

In [48]:
def accuracy_calculator(true_predictions, true_labels):
  accuracy = []
  for pred, label in zip(true_predictions,true_labels):
      counter = 0
      total_tokens = 0
      for i in range(len(label)):
        if pred[i]==label[i]:
          counter += 1 
          total_tokens +=1
        else:
          total_tokens += 1 
      accuracy.append(counter/total_tokens)
  return accuracy

In [49]:
accuracy_distilled_bert = accuracy_calculator(true_predictions, true_labels)

In [50]:
accuracy_distilled_bert_df = pd.DataFrame({'accuracy_distilled_bert':accuracy_distilled_bert})

In [51]:
accuracy_distilled_bert_df.to_csv('accuracy_distilled_bert_df.csv', index=False)

In [52]:

def get_tokens_from_predictions(predictions):
  predicted_tokens= []
  for i in range(len(predictions)):
    prediction_arr = predictions[i]
    used_tokens = tokenizer.tokenize(tokenized_datasets['test']['text'][i])
    if(len(prediction_arr)-2 == len(used_tokens)):
      prediction_arr = prediction_arr[1:-1]
    elif(len(prediction_arr)-1 == len(used_tokens)):
      prediction_arr = prediction_arr[1:]
    elif(len(used_tokens)> len(prediction_arr)):
      for t in range(len(used_tokens)- len(prediction_arr)):
        prediction_arr.append(0)
    elif(len(used_tokens)< len(prediction_arr)):
        prediction_arr = prediction_arr[len(prediction_arr)-len(used_tokens)-1:-1]
    if(len(prediction_arr) != len(used_tokens)):
      print(len(prediction_arr), len(used_tokens))
    toxic_tokens = []
    for j in range(len(prediction_arr)):
      if(prediction_arr[j] == 1):
        if used_tokens[j][0] == "Ġ":
          used_tokens[j] = used_tokens[j][1:]
        toxic_tokens.append(used_tokens[j])
    predicted_tokens.append(toxic_tokens)
  
  return predicted_tokens

In [53]:
predicted_tokens_spans= get_tokens_from_predictions(true_predictions)

In [55]:
def prediction_to_spans(predicted_tokens):
  predicted_spans= []
  for i in range(len(predicted_tokens)):
    curr_tokens = predicted_tokens[i]
    text = tokenized_datasets['test']['text'][i]
    #print(curr_tokens)
    #print(text)
    spans = []
    for item in curr_tokens:
      idx = -1
      if item not in text:
        continue
      if len(spans) == 0:
        idx = text.index(item)
      else:
        #print(item not in text)
        #print(item)
        if(item == 'sc'):
          text = text.lower()
        id = spans[len(spans)-1]
        if item in text[id:]:
          idx = text.index(item, spans[len(spans)-1])
      if(idx != -1):
        for j in range(len(item)):
          spans.append(idx)
          idx = idx + 1
    predicted_spans.append(spans)
  
  return predicted_spans

In [56]:
predicted_spans = prediction_to_spans(predicted_tokens_spans)

In [57]:
file=open('span_predictions_distil.txt','w')
for items in predicted_spans:
    if len(items) == 0:
      file.writelines('[]'+'\n')
    else:
      s = ''
      for i in range(len(items)):
        s = s + str(items[i]) + ','

      file.writelines('['+s[:-1]+ ']'+'\n')
file.close()